# OBJETIVO
* Análise exploratória dos dados com os seguintes fins:
    * Identificação das localidades onde cada modelo tem os maiores preços
    * Verificação da quilometragem ótima de venda

# Carregamento de pacotes

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings

# Configuração do notebook

In [ ]:
warnings.filterwarnings('ignore')
%matplotlib inline

# Funções

In [ ]:
# Função para montagem do histograma
def show_outliers_boxplots(df, info):
    Q1 = df[info].quantile(0.25)
    Q3 = df[info].quantile(0.75)
    IQR = Q3 - Q1
    lower_lim = Q1 - 1.5 * IQR
    upper_lim = Q3 + 1.5 * IQR
    ancor = ' -> 0' if lower_lim < 0 else ''
    print(f'lower limit = {lower_lim}{ancor}')
    print(f'upper limit = {upper_lim}')
    limits = [lower_lim, upper_lim]
    df_clean = df[(df[info] >= lower_lim) & (df[info] <= upper_lim)]

    fig, axis = plt.subplots(nrows=2)
    plt.subplots_adjust(hspace = 1)
    sns.boxplot(df_clean[info], ax=axis[0]).set_title('COM outliers')
    sns.boxplot(df_clean[info], ax=axis[1]).set_title('SEM outliers')
    plt.show()

    return limits, df_clean


def plot(df, info, type = 'lineplot', sort=False, xtick_rotation=30, color='lightblue'):
    fig, ax = plt.subplots()
    if sort:
        df_sorted = df.sort_values(by=info)
    else:
        df_sorted = df
    exec(f"sns.{type}(df_sorted.index, df_sorted[info], color='{color}')")
    plt.xticks(rotation=xtick_rotation, ha='right')
    plt.show()


def relative_value(df, base_index, info):
    base_value = df[info][base_index]
    col_name = f'{info}_DiferençaRelativa{base_index}_(%)'
    df[col_name] = (df[info]/base_value)*100 -100
    return df, col_name


# Carregamento da base de dados

In [ ]:
df_full = pd.read_csv('base_tratada.csv')
df_full.info()

In [ ]:
df_full.drop('Unnamed: 0', axis=1, inplace=True)
df_full['VolumeMotor'] = df_full['VolumeMotor'].apply(str)
df_full.info()

# Coleta de amostra da base de dados

In [ ]:
df = df_full
# df = df_full.head(10)
# df = df_full.sample(50)

df.head()

# Análise dos dados com base no modelo

In [ ]:
models = np.sort(df['Modelo'].unique())

print('MODELOS')
print(models)

In [ ]:
# Escolha do modelo a ser analisado
model_choice = ['S10']

df_filtered = df[df['Modelo'].isin(model_choice)]
df_filtered

In [ ]:
# Oções de combustível
fuels = np.sort(df_filtered['Combustivel'].unique())

print('COMBUSTÍVEIS')
print(fuels)

In [ ]:
fuel_choice = ['DIESEL']

df_fuel_filtered = df_filtered[df['Combustivel'].isin(fuel_choice)]
df_fuel_filtered

# Análise de outliers

In [ ]:
# Outliers de quilometragem
info = 'Quilometragem'
limits_km = show_outliers_boxplots(df_fuel_filtered, info)
df_fuel_filtered = limits_km[1] # retira os outliers de km da base

In [ ]:
# Outliers de preço anunciado

info = 'PrecoAnuncio'
limits_price = show_outliers_boxplots(df_fuel_filtered, info)
df_fuel_filtered = limits_price[1] # retira os outliers de preço da base


# Análise influência das versões e do ano do modelo sobre o preço

In [ ]:
df_fuel_filtered.head()

In [ ]:
group = 'Descricao'
info = 'PrecoAnuncio'
df_group = df_fuel_filtered.groupby([group]).mean()
plot(df_group, info, sort=True, type='barplot')

In [ ]:
group = 'AnoModelo'
info = 'PrecoAnuncio'
df_group = df_fuel_filtered.groupby([group]).mean()
plot(df_group, info, type='barplot')

# Análise da variação de preço por estado

In [ ]:
# Variação de preço por estado
base_index = 'RJ' # Estado referência para cálculo do preço relativo

group = 'Estado'
info = 'PrecoAnuncio'
df_group = df_fuel_filtered.groupby([group]).mean()

df_relative = df_group
(df_relative_price, relative_price_col) = relative_value(df_relative, base_index, info)

plot(df_relative_price, relative_price_col, type='barplot', sort=True, xtick_rotation=90)

# Análise da variação de preço por tipo de transmissão

In [ ]:
# Verificação dos tipos de transmissão disponíveis e cálculo das médias
group = 'Trasmissao'
info = 'PrecoAnuncio'
df_group = df_fuel_filtered.groupby([group]).mean()
print(df_group.index.values)

In [ ]:
# Variação de preço por tipo de transmissão
base_index = 'MANUAL' # Escolha do tipo de transmissão de referência

df_relative = df_group
(df_relative_price, relative_price_col) = relative_value(df_relative, base_index, info)

plot(df_relative_price, relative_price_col, type='barplot', sort=True)

# Análise da relação do preço com a quilometragem
## Como esta análise trata de diferenças de peço mais sutis, é relevante analisar cada versão do modelo escolhido separadamente

In [ ]:
km_min = df_fuel_filtered['Quilometragem'].min()
km_max = df_fuel_filtered['Quilometragem'].max()
print(km_min)
print(km_max)

In [ ]:
km_bins = [0]
interval = 10000

km_initial = km_bins[0]
while km_initial <= km_max:
    km_bins.append(max(km_bins) + interval)
    km_initial = max(km_bins)

km_bins_labels = []
for i in range(0, len(km_bins)-1):
    km_bins_labels.append(f'{int(km_bins[i]/1000)} - {int(km_bins[i+1]/1000)}')
km_bins_labels

In [ ]:
model_versions = df_fuel_filtered['Descricao'].unique()
model_versions

In [ ]:
df_price_by_km = pd.DataFrame()
df_price_by_km.index = model_versions
# df_price_by_km

In [ ]:
for i in range(0, len(km_bins_labels)):
    means = []
    for version in model_versions:
        mean = df_fuel_filtered[(df_fuel_filtered['Descricao'] == version) &
                           (df_fuel_filtered['Quilometragem'] >= km_bins[i]) &
                           (df_fuel_filtered['Quilometragem'] < km_bins[i + 1])].mean()['PrecoAnuncio']
        means.append(mean)    
    df_price_by_km[km_bins_labels[i]] = means

# df_price_by_km

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.lineplot(data = df_price_by_km.transpose(), palette='dark')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.title('Preço x Quilometragem')
plt.xlabel('Mil Quilômetros')
plt.ylabel('reço (R$)')
plt.xticks(rotation=30, ha='right')
plt.show()